In [532]:
import pandas as pd
import os
import re
from sklearn.preprocessing import OneHotEncoder

In [533]:
data = pd.read_csv('<file path>/<file name>.csv')
data.head()

/Users/briannahancock/opt/anaconda3/envs/PythonData2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [534]:
# Dropping rows with bad data
data = data.drop(labels=[19729,19730,29502,29503,35586,35587], axis=0)

In [535]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45460 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45460 non-null  object 
 1   belongs_to_collection  4490 non-null   object 
 2   budget                 45460 non-null  object 
 3   genres                 45460 non-null  object 
 4   homepage               7778 non-null   object 
 5   id                     45460 non-null  object 
 6   imdb_id                45443 non-null  object 
 7   original_language      45449 non-null  object 
 8   original_title         45460 non-null  object 
 9   overview               44506 non-null  object 
 10  popularity             45460 non-null  object 
 11  poster_path            45077 non-null  object 
 12  production_companies   45460 non-null  object 
 13  production_countries   45460 non-null  object 
 14  release_date           45376 non-null  object 
 15  re

In [536]:
# Dropping columns with lots of null rows, and JSON data types
data = data.drop(columns=['belongs_to_collection', 'homepage', 'overview', 'production_companies', 'production_countries', 'spoken_languages', 'tagline'])

In [537]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45460 entries, 0 to 45465
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              45460 non-null  object 
 1   budget             45460 non-null  object 
 2   genres             45460 non-null  object 
 3   id                 45460 non-null  object 
 4   imdb_id            45443 non-null  object 
 5   original_language  45449 non-null  object 
 6   original_title     45460 non-null  object 
 7   popularity         45460 non-null  object 
 8   poster_path        45077 non-null  object 
 9   release_date       45376 non-null  object 
 10  revenue            45460 non-null  float64
 11  runtime            45203 non-null  float64
 12  status             45379 non-null  object 
 13  title              45460 non-null  object 
 14  video              45460 non-null  object 
 15  vote_average       45460 non-null  float64
 16  vote_count         454

In [538]:
data.columns

Index(['adult', 'budget', 'genres', 'id', 'imdb_id', 'original_language',
       'original_title', 'popularity', 'poster_path', 'release_date',
       'revenue', 'runtime', 'status', 'title', 'video', 'vote_average',
       'vote_count'],
      dtype='object')

In [539]:
imdb = pd.Series(data.imdb_id)
imdb

0        tt0114709
1        tt0113497
2        tt0113228
3        tt0114885
4        tt0113041
           ...    
45461    tt6209470
45462    tt2028550
45463    tt0303758
45464    tt0008536
45465    tt6980792
Name: imdb_id, Length: 45460, dtype: object

In [540]:
# Cleaning up imdb id to map to other tables
new_imdb = []
for id in imdb:
    # print(id)
    id = str(id)
    id = id.replace('tt', '')
    new_imdb.append(id)
new_imdb

['0114709',
 '0113497',
 '0113228',
 '0114885',
 '0113041',
 '0113277',
 '0114319',
 '0112302',
 '0114576',
 '0113189',
 '0112346',
 '0112896',
 '0112453',
 '0113987',
 '0112760',
 '0112641',
 '0114388',
 '0113101',
 '0112281',
 '0113845',
 '0113161',
 '0112722',
 '0112401',
 '0114168',
 '0113627',
 '0114057',
 '0114011',
 '0114117',
 '0112682',
 '0115012',
 '0112792',
 '0114746',
 '0114952',
 '0112431',
 '0112637',
 '0112818',
 '0112286',
 '0113442',
 '0112697',
 '0112749',
 '0114279',
 '0112819',
 '0114272',
 '0113855',
 '0114681',
 '0113347',
 '0114369',
 '0114148',
 '0114916',
 '0114814',
 '0109950',
 '0113819',
 '0110299',
 '0112499',
 '0113158',
 '0113541',
 '0113321',
 '0110877',
 '0112714',
 '0113419',
 '0116260',
 '0113862',
 '0116126',
 '0118002',
 '0115683',
 '0116839',
 '0114753',
 '0113149',
 '0113118',
 '0116367',
 '0113010',
 '0113537',
 '0113828',
 '0115644',
 '0115676',
 '0114367',
 '0113973',
 '0112744',
 '0116731',
 '0112445',
 '0114660',
 '0112379',
 '0114039',
 '01

In [541]:
data['imdb_id'] = new_imdb

In [542]:
# Dropping imdb_ids that are not numeric
data = data[data.imdb_id.apply(lambda x: x.isnumeric())]

In [543]:
data

,adult,budget,genres,id,imdb_id,original_language,original_title,popularity,poster_path,release_date,revenue,runtime,status,title,video,vote_average,vote_count
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,0114709,en,Toy Story,21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,Toy Story,False,7.7,5415.0
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,0113497,en,Jumanji,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Jumanji,False,6.9,2413.0
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,0113228,en,Grumpier Old Men,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Grumpier Old Men,False,6.5,92.0
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,0114885,en,Waiting to Exhale,3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,False,6.1,34.0
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",11862,0113041,en,Father of the Bride Part II,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",439050,6209470,fa,رگ خواب,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,NaN,0.0,90.0,Released,Subdue,False,4.0,1.0
45462,False,0,"[{'id': 18, 'name': 'Drama'}]",111109,2028550,tl,Siglo ng Pagluluwal,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,2011-11-17,0.0,360.0,Released,Century of Birthing,False,9.0,3.0
45463,False,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,0303758,en,Betrayal,0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,2003-08-01,0.0,90.0,Released,Betrayal,False,3.8,6.0
45464,False,0,[],227506,0008536,en,Satana likuyushchiy,0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,1917-10-21,0.0,87.0,Released,Satan Triumphant,False,0.0,0.0


In [544]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45443 entries, 0 to 45465
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              45443 non-null  object 
 1   budget             45443 non-null  object 
 2   genres             45443 non-null  object 
 3   id                 45443 non-null  object 
 4   imdb_id            45443 non-null  object 
 5   original_language  45432 non-null  object 
 6   original_title     45443 non-null  object 
 7   popularity         45443 non-null  object 
 8   poster_path        45065 non-null  object 
 9   release_date       45362 non-null  object 
 10  revenue            45443 non-null  float64
 11  runtime            45188 non-null  float64
 12  status             45362 non-null  object 
 13  title              45443 non-null  object 
 14  video              45443 non-null  object 
 15  vote_average       45443 non-null  float64
 16  vote_count         454

In [545]:
data['id'].nunique()

45413

In [546]:
data['original_language'].nunique()

89

In [547]:
data['original_language'].value_counts()

en    32254
fr     2437
it     1529
ja     1348
de     1080
      ...  
ay        1
rw        1
cy        1
tg        1
si        1
Name: original_language, Length: 89, dtype: int64

In [548]:
# Dropping duplicate movie id's for primary key constraint
data = data.drop_duplicates(subset='id', keep='first')

In [549]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45413 entries, 0 to 45465
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              45413 non-null  object 
 1   budget             45413 non-null  object 
 2   genres             45413 non-null  object 
 3   id                 45413 non-null  object 
 4   imdb_id            45413 non-null  object 
 5   original_language  45402 non-null  object 
 6   original_title     45413 non-null  object 
 7   popularity         45413 non-null  object 
 8   poster_path        45035 non-null  object 
 9   release_date       45332 non-null  object 
 10  revenue            45413 non-null  float64
 11  runtime            45158 non-null  float64
 12  status             45332 non-null  object 
 13  title              45413 non-null  object 
 14  video              45413 non-null  object 
 15  vote_average       45413 non-null  float64
 16  vote_count         454

In [550]:
data.head()

,adult,budget,genres,id,imdb_id,original_language,original_title,popularity,poster_path,release_date,revenue,runtime,status,title,video,vote_average,vote_count
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,0114709,en,Toy Story,21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,Toy Story,False,7.7,5415.0
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,0113497,en,Jumanji,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Jumanji,False,6.9,2413.0
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,0113228,en,Grumpier Old Men,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Grumpier Old Men,False,6.5,92.0
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,0114885,en,Waiting to Exhale,3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,False,6.1,34.0
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",11862,0113041,en,Father of the Bride Part II,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,False,5.7,173.0


In [551]:
# Regex to extract the names of the genres in JSON formatted data
data['list_genres'] = data['genres'].str.findall("(?:'name':\s)(\W[A-Z][a-z]*\W)(?:})")
data.head()

/Users/briannahancock/opt/anaconda3/envs/PythonData2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,adult,budget,genres,id,imdb_id,original_language,original_title,popularity,poster_path,release_date,revenue,runtime,status,title,video,vote_average,vote_count,list_genres
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,0114709,en,Toy Story,21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,Toy Story,False,7.7,5415.0,"['Animation', 'Comedy', 'Family']"
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,0113497,en,Jumanji,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Jumanji,False,6.9,2413.0,"['Adventure', 'Fantasy', 'Family']"
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,0113228,en,Grumpier Old Men,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Grumpier Old Men,False,6.5,92.0,"['Romance', 'Comedy']"
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,0114885,en,Waiting to Exhale,3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,False,6.1,34.0,"['Comedy', 'Drama', 'Romance']"
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",11862,0113041,en,Father of the Bride Part II,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,False,5.7,173.0,['Comedy']


In [552]:
# Replacing commas from csv for importing into postgres database
titles = data['title']
cleaned_titles = []
for title in titles:
    title = str(title)
    title = title.replace(',', ':')
    cleaned_titles.append(title)
cleaned_titles

['Toy Story',
 'Jumanji',
 'Grumpier Old Men',
 'Waiting to Exhale',
 'Father of the Bride Part II',
 'Heat',
 'Sabrina',
 'Tom and Huck',
 'Sudden Death',
 'GoldenEye',
 'The American President',
 'Dracula: Dead and Loving It',
 'Balto',
 'Nixon',
 'Cutthroat Island',
 'Casino',
 'Sense and Sensibility',
 'Four Rooms',
 'Ace Ventura: When Nature Calls',
 'Money Train',
 'Get Shorty',
 'Copycat',
 'Assassins',
 'Powder',
 'Leaving Las Vegas',
 'Othello',
 'Now and Then',
 'Persuasion',
 'The City of Lost Children',
 'Shanghai Triad',
 'Dangerous Minds',
 'Twelve Monkeys',
 'Wings of Courage',
 'Babe',
 'Carrington',
 'Dead Man Walking',
 'Across the Sea of Time',
 'It Takes Two',
 'Clueless',
 'Cry: the Beloved Country',
 'Richard III',
 'Dead Presidents',
 'Restoration',
 'Mortal Kombat',
 'To Die For',
 'How To Make An American Quilt',
 'Se7en',
 'Pocahontas',
 'When Night Is Falling',
 'The Usual Suspects',
 'Guardian Angel',
 'Mighty Aphrodite',
 'Lamerica',
 'The Big Green',
 'Geo

In [553]:
data['title'] = cleaned_titles

/Users/briannahancock/opt/anaconda3/envs/PythonData2/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [554]:
data.head()

,adult,budget,genres,id,imdb_id,original_language,original_title,popularity,poster_path,release_date,revenue,runtime,status,title,video,vote_average,vote_count,list_genres
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,0114709,en,Toy Story,21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,Toy Story,False,7.7,5415.0,"['Animation', 'Comedy', 'Family']"
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,0113497,en,Jumanji,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Jumanji,False,6.9,2413.0,"['Adventure', 'Fantasy', 'Family']"
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,0113228,en,Grumpier Old Men,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Grumpier Old Men,False,6.5,92.0,"['Romance', 'Comedy']"
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,0114885,en,Waiting to Exhale,3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,False,6.1,34.0,"['Comedy', 'Drama', 'Romance']"
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",11862,0113041,en,Father of the Bride Part II,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,False,5.7,173.0,['Comedy']


In [555]:
# creating list of all genres to encode
from ast import literal_eval
ind, value = zip(*[(i, x) 
                  for i, row in data['genres'].items() 
                  for x in literal_eval(row)])
genre_col = pd.DataFrame(value, index=ind)
genre_col

,id,name
0,16,Animation
0,35,Comedy
0,10751,Family
1,12,Adventure
1,14,Fantasy
...,...,...
45461,10751,Family
45462,18,Drama
45463,28,Action
45463,18,Drama


In [556]:
# Dropping replaced columns
data = data.drop(columns=['genres', 'list_genres', 'original_title'])
data.head()

,adult,budget,id,imdb_id,original_language,popularity,poster_path,release_date,revenue,runtime,status,title,video,vote_average,vote_count
0,False,30000000,862,0114709,en,21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,Toy Story,False,7.7,5415.0
1,False,65000000,8844,0113497,en,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Jumanji,False,6.9,2413.0
2,False,0,15602,0113228,en,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Grumpier Old Men,False,6.5,92.0
3,False,16000000,31357,0114885,en,3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,False,6.1,34.0
4,False,0,11862,0113041,en,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,False,5.7,173.0


In [557]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45413 entries, 0 to 45465
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              45413 non-null  object 
 1   budget             45413 non-null  object 
 2   id                 45413 non-null  object 
 3   imdb_id            45413 non-null  object 
 4   original_language  45402 non-null  object 
 5   popularity         45413 non-null  object 
 6   poster_path        45035 non-null  object 
 7   release_date       45332 non-null  object 
 8   revenue            45413 non-null  float64
 9   runtime            45158 non-null  float64
 10  status             45332 non-null  object 
 11  title              45413 non-null  object 
 12  video              45413 non-null  object 
 13  vote_average       45413 non-null  float64
 14  vote_count         45413 non-null  float64
dtypes: float64(4), object(11)
memory usage: 5.5+ MB


In [558]:
data.to_csv('<file path>/<file name>.csv')

In [559]:
# Renaming columns for encoding
genre_col = genre_col.rename(columns={'name':'genre'})
genre_col

,id,genre
0,16,Animation
0,35,Comedy
0,10751,Family
1,12,Adventure
1,14,Fantasy
...,...,...
45461,10751,Family
45462,18,Drama
45463,28,Action
45463,18,Drama


In [560]:
genre_col['genre'].value_counts()

Drama              20239
Comedy             13173
Thriller            7616
Romance             6728
Action              6587
Horror              4670
Crime               4303
Documentary         3929
Adventure           3487
Science Fiction     3041
Family              2767
Mystery             2463
Fantasy             2307
Animation           1927
Foreign             1617
Music               1597
History             1397
War                 1322
Western             1041
TV Movie             765
Name: genre, dtype: int64

In [561]:
# Dropping genres that contain publisher info in genres column
genre_col['count'] = genre_col.groupby('genre').transform('count')
genre_counts = genre_col.loc[genre_col['count'] >= 2].drop(columns='count')

In [562]:
genre_counts['genre'].value_counts()

Drama              20239
Comedy             13173
Thriller            7616
Romance             6728
Action              6587
Horror              4670
Crime               4303
Documentary         3929
Adventure           3487
Science Fiction     3041
Family              2767
Mystery             2463
Fantasy             2307
Animation           1927
Foreign             1617
Music               1597
History             1397
War                 1322
Western             1041
TV Movie             765
Name: genre, dtype: int64

In [563]:
genre_counts

,id,genre
0,16,Animation
0,35,Comedy
0,10751,Family
1,12,Adventure
1,14,Fantasy
...,...,...
45461,10751,Family
45462,18,Drama
45463,28,Action
45463,18,Drama


In [564]:
# Create list of all columns to be encoded
genre_encoding_list = genre_counts.dtypes[genre_counts.dtypes == "object"].index.tolist()
genre_encoding_list

['genre']

In [565]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encoded_genres = pd.DataFrame(enc.fit_transform(genre_counts[genre_encoding_list]), index=genre_counts.index)

# Add the encoded variable names to the dataframe
encoded_genres.columns = enc.get_feature_names_out(genre_encoding_list)
encoded_genres.head()

,genre_Action,genre_Adventure,genre_Animation,genre_Comedy,genre_Crime,genre_Documentary,genre_Drama,genre_Family,genre_Fantasy,genre_Foreign,genre_History,genre_Horror,genre_Music,genre_Mystery,genre_Romance,genre_Science Fiction,genre_TV Movie,genre_Thriller,genre_War,genre_Western
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [582]:
encoded_genres.isna().sum()

genre_action             0
genre_adventure          0
genre_animation          0
genre_comedy             0
genre_crime              0
genre_documentary        0
genre_drama              0
genre_family             0
genre_fantasy            0
genre_foreign            0
genre_history            0
genre_horror             0
genre_music              0
genre_mystery            0
genre_romance            0
genre_science fiction    0
genre_tv movie           0
genre_thriller           0
genre_war                0
genre_western            0
dtype: int64

In [566]:
encoded_genres.columns

Index(['genre_Action', 'genre_Adventure', 'genre_Animation', 'genre_Comedy',
       'genre_Crime', 'genre_Documentary', 'genre_Drama', 'genre_Family',
       'genre_Fantasy', 'genre_Foreign', 'genre_History', 'genre_Horror',
       'genre_Music', 'genre_Mystery', 'genre_Romance',
       'genre_Science Fiction', 'genre_TV Movie', 'genre_Thriller',
       'genre_War', 'genre_Western'],
      dtype='object')

In [567]:
# Summing columns to group duplicated indexes
encoded_genres = encoded_genres.groupby(encoded_genres.index)['genre_Action', 'genre_Adventure', 'genre_Animation', 'genre_Comedy',
       'genre_Crime', 'genre_Documentary', 'genre_Drama', 'genre_Family',
       'genre_Fantasy', 'genre_Foreign', 'genre_History', 'genre_Horror',
       'genre_Music', 'genre_Mystery', 'genre_Romance',
       'genre_Science Fiction', 'genre_TV Movie', 'genre_Thriller',
       'genre_War', 'genre_Western'].apply(lambda x : x.astype(int).sum())

/Users/briannahancock/opt/anaconda3/envs/PythonData2/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [568]:
encoded_genres.head()

,genre_Action,genre_Adventure,genre_Animation,genre_Comedy,genre_Crime,genre_Documentary,genre_Drama,genre_Family,genre_Fantasy,genre_Foreign,genre_History,genre_Horror,genre_Music,genre_Mystery,genre_Romance,genre_Science Fiction,genre_TV Movie,genre_Thriller,genre_War,genre_Western
0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [569]:
# Column names to lower for SQL
encoded_genres.columns = encoded_genres.columns.str.lower()

In [570]:
encoded_genres.head()

,genre_action,genre_adventure,genre_animation,genre_comedy,genre_crime,genre_documentary,genre_drama,genre_family,genre_fantasy,genre_foreign,genre_history,genre_horror,genre_music,genre_mystery,genre_romance,genre_science fiction,genre_tv movie,genre_thriller,genre_war,genre_western
0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [583]:
encoded_genres.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42976 entries, 0 to 45463
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   genre_action           42976 non-null  int64
 1   genre_adventure        42976 non-null  int64
 2   genre_animation        42976 non-null  int64
 3   genre_comedy           42976 non-null  int64
 4   genre_crime            42976 non-null  int64
 5   genre_documentary      42976 non-null  int64
 6   genre_drama            42976 non-null  int64
 7   genre_family           42976 non-null  int64
 8   genre_fantasy          42976 non-null  int64
 9   genre_foreign          42976 non-null  int64
 10  genre_history          42976 non-null  int64
 11  genre_horror           42976 non-null  int64
 12  genre_music            42976 non-null  int64
 13  genre_mystery          42976 non-null  int64
 14  genre_romance          42976 non-null  int64
 15  genre_science fiction  42976 non-nul

In [571]:
encoded_genres.to_csv('<file path>/<file name>.csv')

In [572]:
movies = pd.read_csv('<file path>/<file name>.csv', index_col='movieId')
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [573]:
# Replacing commas for SQL database
movie_titles = movies['title']
cleaned_movies = []
for movie in movie_titles:
    movie = str(movie)
    movie = movie.replace(',', ';')
    cleaned_movies.append(movie)
cleaned_movies

['Toy Story (1995)',
 'Jumanji (1995)',
 'Grumpier Old Men (1995)',
 'Waiting to Exhale (1995)',
 'Father of the Bride Part II (1995)',
 'Heat (1995)',
 'Sabrina (1995)',
 'Tom and Huck (1995)',
 'Sudden Death (1995)',
 'GoldenEye (1995)',
 'American President; The (1995)',
 'Dracula: Dead and Loving It (1995)',
 'Balto (1995)',
 'Nixon (1995)',
 'Cutthroat Island (1995)',
 'Casino (1995)',
 'Sense and Sensibility (1995)',
 'Four Rooms (1995)',
 'Ace Ventura: When Nature Calls (1995)',
 'Money Train (1995)',
 'Get Shorty (1995)',
 'Copycat (1995)',
 'Assassins (1995)',
 'Powder (1995)',
 'Leaving Las Vegas (1995)',
 'Othello (1995)',
 'Now and Then (1995)',
 'Persuasion (1995)',
 'City of Lost Children; The (Cité des enfants perdus; La) (1995)',
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 'Dangerous Minds (1995)',
 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',
 'Wings of Courage (1995)',
 'Babe (1995)',
 'Carrington (1995)',
 'Dead Man Walking (1995)',
 'Across the Sea of 

In [574]:
movies['title'] = cleaned_movies

In [575]:
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [576]:
movies.to_csv('<file path>/<file name>.csv')

In [577]:
tag_data = pd.read_csv('<file path>/<file name>.csv', index_col='movieId')
tag_data.head()

,userId,tag,timestamp
movieId,,,
260,3,classic,1439472355
260,3,sci-fi,1439472256
1732,4,dark comedy,1573943598
1732,4,great dialogue,1573943604
7569,4,so bad it's good,1573943455


In [578]:
# Removing commas for SQL database
tag_tags = tag_data['tag']
cleaned_tags = []
for tag in tag_tags:
    tag = str(tag)
    tag = tag.replace(',', '')
    cleaned_tags.append(tag)
cleaned_tags

['classic',
 'sci-fi',
 'dark comedy',
 'great dialogue',
 "so bad it's good",
 'unreliable narrators',
 'tense',
 'artificial intelligence',
 'philosophical',
 'tense',
 "so bad it's good",
 'cliche',
 'musical',
 'horror',
 'unpredictable',
 'Oscar (Best Supporting Actress)',
 'adventure',
 'anime',
 'ecology',
 'fantasy',
 'Hayao Miyazaki',
 'Miyazaki',
 'post-apocalyptic',
 'bah',
 'Clint Eastwood',
 'music',
 'art',
 'contemporary art',
 'documentary',
 'aliens',
 'amazing photography',
 'Director: James Cameron',
 'first contact',
 'James Cameron',
 'Michael Biehn',
 'sci-fi',
 'android(s)/cyborg(s)',
 'apocalypse',
 'Arnold Schwarzenegger',
 'artificial intelligence',
 'franchise',
 'terminator',
 'time travel',
 'James Cameron',
 'sci-fi',
 'science fiction',
 'sci-fi',
 'inferior sequel',
 'setting:London (UK) (future)',
 'unoriginal',
 'bad science',
 'good science',
 'Hans Zimmer',
 'philosophical issues',
 'sci-fi',
 'science fiction',
 'space',
 'space travel',
 'time-trav

In [579]:
tag_data['tag'] = cleaned_tags

In [580]:
tag_data.head()

,userId,tag,timestamp
movieId,,,
260,3,classic,1439472355
260,3,sci-fi,1439472256
1732,4,dark comedy,1573943598
1732,4,great dialogue,1573943604
7569,4,so bad it's good,1573943455


In [581]:
tag_data.to_csv('<file path>/<file name>.csv')